# Make Grid

<div class="alert alert-block alert-warning">
<b>WARNING:</b> This notebook contains operations instructions for experienced users. 
</div>

<div class="alert alert-block alert-warning">
<b>WARNING:</b> Some instructions on this notebook will be deprecated as of .
</div>

This notebook is intended to aid the construction of a first pointing model with the Auxiliary Telescope. 

At this early stages, and given the small field-of-view of the telescope, it is really hard to get a start on the field. To help on the process, this notebook implements a Gridding routine. The routine will scan around the current position of the telescope and waits for the user to specify if is should continue, go to the next or previous pointing or stop. 



In [ ]:
import logging
import yaml

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio

# import palpy

from lsst.ts import salobj

from lsst.ts.standardscripts.auxtel.attcs import ATTCS

from lsst.ts.idl.enums import ATPtg

%matplotlib inline

## Disable auto download of iers data. 

Updated IERS data is needed in order to provide proper coordinates transformation. By default astropy will try to update the information when we create a new ICRS or KF5 coordinate object. The next cell will disable auto download to speed up the container in case there are network access issues. Trying to download data when there is no outside access results in a long wait time until the connection times out. 

In [ ]:
from astropy.utils import iers
iers.conf.auto_download = False

## Check value of the `LSST_DDS_DOMAIN` variable.

For the AT early work at the summit, the expected value for `LSST_DDS_DOMAIN=lsatmcs`

In [ ]:
import os
print(os.environ["LSST_DDS_DOMAIN"])

## Create a domain and remotes

The ATTCS class uses the following remotes:

  - atmcs 
  - atptg
  - ataos
  - atpneumatics 
  - athexapod
  - atdome
  - atdometrajectory


In [ ]:
d = salobj.Domain()

In case, you need to build the idl files, copy the following command to a cell and run it. 

```
%%script bash 
make_idl_files.py ATMCS ATPtg ATAOS ATPneumatics ATHexapod ATDome ATDomeTrajectory
```

In [ ]:
atmcs = salobj.Remote(d, "ATMCS")
atptg = salobj.Remote(d, "ATPtg")

In [ ]:
await asyncio.gather(atmcs.start_task, 
                     atptg.start_task)

In [ ]:
attcs = ATTCS(atmcs=atmcs, 
              atptg=atptg, 
              ataos=ataos, 
              atpneumatics=atpne, 
              athexapod=athex, 
              atdome=atdome, 
              atdometrajectory=atdomtraj, 
              check={"atpneumatics": False, "athexapod": False, "atdome": True, "atdometrajectory": True})

# Slew and Track an Alt/Az position

While the pointing component does not support the slew and track of an alt/az position, the following cells will provide a quick and easy way to perform this task. The user specified a position in Alt/Az and it will use astropy coordinate library to convert it to RA/Dec. 

For that we will need the location of the observatory and the time. Location is defined as an astropy `EarthLocation` and time is taken from the pointing component and then creating an astropy `Time` object.

Obervatory location (lon, lat and height extracted from: https://github.com/lsst/sims_utils)

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
alt = 70. * u.deg
az = 0. * u.deg

The next cell will convert the specified Alt/Az into RA/Dec coordinates in ICRS. This coordinate will be used to slew and track. 

In [ ]:
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_atptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_atptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

await attcs.slew(ra=cmd_radec.ra.hour, 
                 dec=cmd_radec.dec.deg, 
                 rotPA=180.-cmd_elaz.alt.deg,
                 rot_frame=ATPtg.RotFrame.FIXED,
                 rot_mode=ATPtg.RotMode.FIELD)

print(f"raDecTarget ra={cmd_radec.ra!r} hour; "
      f"declination={cmd_radec.dec!r} deg")
# script.ataos.cmd_enableCorrection.set(hexapod=True)
# await script.ataos.cmd_enableCorrection.start(timeout=10)



# Slew and track a RA/Dec target

The next cell shows an example of how to slew and track an RA/Dec target. 

The one caveat with slewing to RA/Dec is getting a proper value for rotPA. We currently want to keep the rotator around zero degrees. We need to improve the handling of this rotation angle on the slew method but right now we compute the angle on the cell. 

In [ ]:
ra = Angle('19:05:24', unit=u.hour)
dec = Angle('+13:51:47', unit=u.deg)
target_name="HR 7235"

radec = ICRS(ra, dec)

# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

await attcs.slew(ra=ra, 
                 dec=dec,
                 rotPA=180.-alt_az.alt.deg,
                 rot_frame=ATPtg.RotFrame.FIXED,
                 rot_mode=ATPtg.RotMode.FIELD)

# Slew and track the MOON (and other solar system body)

In [ ]:
await attcs.slew_to_planet(planet=ATPtg.Planets.MOON)

# GRID in RA/Dec

In [ ]:
dra_grid = np.arange(-1., 1.1,0.1)*24./360.
ddec_grid = np.arange(-1., 1.1,0.1)

In [ ]:
grid_ra = np.zeros(len(dra_grid)*len(ddec_grid))
grid_dec = np.zeros(len(dra_grid)*len(ddec_grid))

for i in range(len(ddec_grid)):
    grid_ra[i*len(dra_grid):(i+1)*len(dra_grid)] += dra_grid[::(-1)**i]
    grid_dec[i*len(dra_grid):(i+1)*len(dra_grid)] += ddec_grid[i]

In [ ]:
plt.plot(grid_ra[0], grid_dec[0], 'bo')
plt.plot(grid_ra, grid_dec, ':')
plt.plot(grid_ra[-1], grid_dec[-1], 'go')

In [ ]:
async def wait_center():
    while True:
        opt = input("Center telescope and choose action (press ?<enter>, for list of actions): ")
        
        if opt == 'n':
            print("Next point in the grid...")
            return 1
        elif opt == 'p':
            print("Previous point in the grid...")
            return -1
        elif opt == 't':
            print("Stopping test...")
            return 0
        elif opt == '?':
            print("""Options are:
            n - Go to next point in the grid.
            p - Go to previous point in the grid.
            t - Terminate test.
            """)
        else:
            print("Next point in the grid...")
            return 1

In [ ]:
ra = Angle("01:37:42.84548", unit=u.hour)
dec = Angle("-57:14:12.3101", unit=u.deg)
target_name="Alpha Eri"

radec = ICRS(ra, dec)

start_from = 0

In [ ]:
await atptg.cmd_stopTracking.start()

In [ ]:
# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
print(curr_time_atptg)
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)

print("slew...")
# await atmcs.cmd_startTracking.start(timeout=10)
await atptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=radec.ra.hour,
    declination=radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=180.-alt_az.alt.deg,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

In [ ]:
start_from = 0

In [ ]:
print(f"Grid has {len(grid_ra)} pointings...")
print(f"Starting from {start_from}")

i = start_from
stop_at = 0

# Figure out what is the rotPA that sets nasmith rotator close to zero.
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_atptg = Time(time_data.tai, format="mjd", scale="tai")
coord_frame_altaz = AltAz(location=location, obstime=curr_time_atptg)
alt_az = radec.transform_to(coord_frame_altaz)


while i < len(grid_ra):
    
    print(f"GRID[{i}]: {grid_ra[i]} x {grid_dec[i]}")

    await attcs.slew(ra=ra.hour + grid_ra[i], 
                     dec=dec.deg + grid_dec[i],
                     rotPA=180.-alt_az.alt.deg,
                     target_name=f"{target_name} RA/Dec GRID[{i}]: {grid_ra[i]} x {grid_dec[i]}",
                     rot_frame=ATPtg.RotFrame.FIXED,
                     rot_mode=ATPtg.RotMode.FIELD)
    
    ret_val = await wait_center()
    if ret_val == 0:
        break

    i += ret_val

    if i < 0:
        print(f"Unwrapping index! (i={i})")
        i = 0


# GRID in Alt/Az

In [ ]:
dalt_grid = np.arange(-0.5, +0.6, 0.1)
daz_grid = np.arange(-0.5, +0.6, 0.1)

In [ ]:
grid_alt = np.zeros(len(dalt_grid)*len(daz_grid))
grid_az = np.zeros(len(dalt_grid)*len(daz_grid))

for i in range(len(daz_grid)):
    grid_alt[i*len(dalt_grid):(i+1)*len(dalt_grid)] += dalt_grid[::(-1)**i]
    grid_az[i*len(dalt_grid):(i+1)*len(dalt_grid)] += daz_grid[i]

In [ ]:
plt.plot(grid_alt, grid_az, ':')
plt.plot(grid_alt, grid_az, '|')
plt.plot(grid_alt[0], grid_az[0], 'bo')
plt.plot(grid_alt[-1], grid_az[-1], 'go')

In [ ]:
start_from=0

In [ ]:
print(f"Grid has {len(grid_alt)} pointings...")
print(f"Starting from {start_from}")

i = start_from
stop_at = 0

while i < len(grid_alt):
    
    print(f"GRID[{i}]: {grid_alt[i]} x {grid_az[i]}")

    await atptg.cmd_offsetAzEl.set_start(el=grid_alt[i]*60.*60., 
                                         az=grid_az[i]*60*60, 
                                         num=0)
    
    ret_val = await wait_center()

    if ret_val == 0:
        break
        
    i += ret_val
    
    if i < 0:
        print(f"Unwrapping index! (i={i})")
        i = 0

In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-0.39*60.*60., 
                                     az=0.25*60.*60., 
                                     num=0)


In [ ]:
await atptg.cmd_offsetAzEl.set_start(el=-0.380*60.*60., 
                                     az=0.251*60.*60., 
                                     num=0)

In [ ]:
await atptg.cmd_pointNewFile.start()
await asyncio.sleep(1.)
await atptg.cmd_pointAddData.start()
await asyncio.sleep(1.)
await atptg.cmd_pointCloseFile.start()

In [ ]:
time_data = await atptg.tel_timeAndDate.next(flush=True, timeout=2)

In [ ]:
time_data.lst

In [ ]:
atdome = salobj.Remote(d, "ATDome")

In [ ]:
await atdome.start_task

In [ ]:
await atdome.cmd_moveAzimuth.set_start(azimuth=65)